In [1]:
import seisbench.data as sbd
from lit_magnet import LitMagNet, prepare_data

model = LitMagNet().load_from_checkpoint(checkpoint_path="../results/model.ckpt")

In [2]:
train_loader, val_loader, test_loader = prepare_data(reshuffle=True, batch_size=128, mode="test")

/home/qgq/miniconda3/lib/python3.8/site-packages/seisbench/data/base.py:1697: DtypeWarning: Columns (7,11,13,14,15,18,19,20,21,22,24,25,26,30,31) have mixed types.Specify dtype option on import or set low_memory=False.
  super().__init__(path=None, name=self._name_internal(), chunks=chunks, **kwargs)


available data size = len(stead)
20641 3440 10321


In [3]:
import pytorch_lightning as pl

trainer = pl.Trainer()
result = trainer.predict(model, test_loader)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
/home/qgq/miniconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:596: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(
The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: ModelCheckpoint
2022-02-25 21:04:22.651920: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-25 21:04:22.651985: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/qgq/miniconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:102: UserWarning: The dataloader, predict dataloader 0, 

Predicting: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 81/81 [03:43<00:00,  2.75s/it]


In [9]:
import torch
y_pred, y_true = [], []
for r in result:
    y_pred.append(r[0].detach())
    y_true.append(r[1].detach())
np_result = torch.cat( (torch.cat(y_pred, 0), torch.cat(y_true, 0)), 1).cpu().numpy()

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [15]:
torch.cat(y_pred, 0).shape, torch.cat(y_true, 0).shape

(torch.Size([10321]), torch.Size([10321, 1]))

In [17]:
import numpy as np
result = np.load('../results/result.npy')

In [18]:
result

array([[ 0.81382453,  1.22      ],
       [ 0.6524671 ,  0.24      ],
       [ 0.65249956,  0.41      ],
       ..., 
       [ 0.65249956,  0.61      ],
       [ 1.0580295 ,  1.41      ],
       [ 0.65249956,  0.33      ]], dtype=float32)

In [19]:
y_pred = result[:, 0]
y_label = result[:, 1]

In [20]:
np.average(np.abs(y_pred - y_label))

0.3279936

In [21]:
np.average(y_pred - y_label)

0.023651885

In [26]:
np.sqrt(np.average(np.square(y_pred - y_label)))

0.43858063

0.40617546999999998